<a href="https://colab.research.google.com/github/wanaylor/RTSP-Object-Detection/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download some COCO Data

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd 'drive/MyDrive/fiftyone'
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/fiftyone
coco-2017/


In [ ]:
%cd '../datasets/detector-recipe'

/content/drive/MyDrive/datasets/detector-recipe


In [ ]:
!pip install fiftyone

In [ ]:
!pip install fiftyone-db-ubuntu2204

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 11.9 MB/s eta 0:00:00


In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

dataset = foz.load_zoo_dataset(
    "coco-2017",
    splits=["validation","train"],
    classes=["person", "car"],
    dataset_name="detector-recipe",
    max_samples=10000,
)

INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/coco-2017/validation' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


Only found 2869 (<10000) samples matching your requirements


Sufficient images already downloaded


...

INFO:fiftyone.utils.coco:Writing annotations for 10000 downloaded samples to '/root/fiftyone/coco-2017/train/labels.json'


Dataset info written to '/root/fiftyone/coco-2017/info.json'


INFO:fiftyone.zoo.datasets:Dataset info written to '/root/fiftyone/coco-2017/info.json'


Loading existing dataset 'detector-recipe'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


INFO:fiftyone.zoo.datasets:Loading existing dataset 'detector-recipe'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [ ]:
dataset

Name:        detector-recipe
Media type:  image
Num samples: 5000
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)

In [ ]:
%ls

coco-2017/  info.json


# Load in the data

In [1]:
from google.colab import drive
drive.mount("/content/drive")
%cd 'drive/MyDrive/datasets/detector-recipe'
%ls

Mounted at /content/drive
/content/drive/MyDrive/datasets/detector-recipe
coco-2017/  info.json


In [2]:
%cd detector-recipe/
%ls

[Errno 2] No such file or directory: 'detector-recipe/'
/content/drive/MyDrive/datasets/detector-recipe
coco-2017/  info.json


In [46]:
import torch
from torchvision.datasets import CocoDetection
from torch.utils.data import DataLoader
from torchvision.transforms import PILToTensor, Resize, Lambda, v2

preproc = v2.Compose([v2.PILToTensor(),
                      v2.Resize((640,640))])


def output_transform(output):
  t = torch.zeros([1,7,100])
  t[:, 4, :] = torch.tensor([1]*100).unsqueeze(dim=0) # initialize everything to background
  for idx, det in enumerate(output):
    if det['category_id'] == 1: #person is 1
      t[:, :, idx] = torch.concat((torch.tensor(det['bbox']), torch.tensor([0,1,0]))).unsqueeze(dim=0)
    if det['category_id'] == 3: #car is 3
      t[:, :, idx] = torch.concat((torch.tensor(det['bbox']), torch.tensor([0,0,1]))).unsqueeze(dim=0)
  return t[0]


dataset = CocoDetection(root='./coco-2017/train/data', annFile='./coco-2017/train/labels.json', transform=preproc, target_transform=output_transform)
train_loader = DataLoader(dataset)

loading annotations into memory...
Done (t=4.13s)
creating index...
index created!


In [43]:
class litedetect(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # Layer 1
        self.a = Conv(in_channels=3, out_channels=8, kernel_size=1, stride=2) # down to 320x320x8
        self.b = Conv(in_channels=8, out_channels=16, kernel_size=1, stride=2) # down to 160x160x16
        self.c = Conv(in_channels=16, out_channels=32, kernel_size=1, stride=2) # down to 80x80x32
        self.d = Conv(in_channels=32, out_channels=64, kernel_size=1, stride=2) # down to 40x40x64
        self.e = Conv(in_channels=64, out_channels=64, kernel_size=1, stride=2) # down to 20x20x64
        self.f = torch.nn.MaxPool2d(kernel_size=(2,2), stride=(1,1), padding=(1,1)) # still 20x20x64

        # Layer 2
        self.f_up = torch.nn.Upsample(size=(40,40)) # 40x40x64
        self.d_l2 = Conv(in_channels=128, out_channels=128, kernel_size=1, stride=1)
        self.d_l2_up = torch.nn.Upsample(size=(80,80)) # 80x80x128
        self.c_l2 = Conv(in_channels=160, out_channels=160, kernel_size=1, stride=1)


    def forward(self, x):
        # Layer 1
        a_out = self.a(x)
        b_out = self.b(a_out)
        c_out = self.c(b_out)
        d_out = self.d(c_out)
        e_out = self.e(d_out)
        f_out = self.f(e_out)

        # Layer 2
        f_up_out = self.f_up(f_out) # 40x40x64
        f_d_cat = torch.concat(f_up_out, d_out) # 40x40x128
        d_l2_out = self.d_l2(f_d_cat)
        d_l2_up_out = self.d_l2_up(d_l2_out) # 80x80x128
        d_c_cat = torch.concat(d_l2_up_out, c_out) # 80x80x160
        c_l2_out = self.c_l2(d_c_cat)


class Conv(torch.nn.Module):
    def  __init__(self,in_channels, out_channels, kernel_size, stride, padding):
        super().__init__()
        self.a = torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size,
                                 stride=stride, padding=padding)
        self.b = torch.nn.BatchNorm2d(num_features=out_channels)
        self.c = torch.nn.SiLU()

    def forward(self,x):
        a_out = self.a(x)
        b_out = self.b(a_out)
        return self.c(b_out

In [ ]:
for data, target in train_loader:
  print(data)
  print(target)
  break

In [45]:
target.shape

torch.Size([1, 7, 100])

In [ ]:
import torch
t = torch.zeros([1,7,100])
t[:, 4, :] = torch.tensor([1]*100).unsqueeze(dim=0)
t[:, :, 1] = torch.concat((torch.tensor(target[1]['bbox']), torch.tensor([0,1,0]))).unsqueeze(0)
t

## Work with YOLOV8 for Loss Function

In [1]:
! git clone https://github.com/ultralytics/ultralytics.git

Cloning into 'ultralytics'...
remote: Enumerating objects: 20418, done.
remote: Counting objects: 100% (1605/1605), done.
remote: Compressing objects: 100% (903/903), done.
remote: Total 20418 (delta 958), reused 1135 (delta 698), pack-reused 18813
Receiving objects: 100% (20418/20418), 11.41 MiB | 13.55 MiB/s, done.
Resolving deltas: 100% (14142/14142), done.


Run this in a python session in the terminal. cd to /content/ultralytics

put a break point in line 203 of loss under utils

In [ ]:
import ultralytics
from ultralytics.models.yolo import model
my_model = model.YOLO('yolov8.yaml')
my_model.train(data="coco128.yaml", epochs=3)